In [1]:
!pip install distance
!pip install fuzzywuzzy

     |████████████████████████████████| 184kB 5.1MB/s 
  Created wheel for distance: filename=Distance-0.1.3-cp37-none-any.whl size=16275 sha256=a377eeea598a1d754f8b3357dbdd949a116fbc6272e74781eecc60c9ba33d77d
  Stored in directory: /root/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e
Successfully built distance


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE
from wordcloud import WordCloud, STOPWORDS
from os import path
from PIL import Image

In [5]:
df = pd.read_csv("new_test.csv")
df.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   test_id    501 non-null    int64 
 1   question1  501 non-null    object
 2   question2  501 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.9+ KB


In [19]:
nan_data = df[df.isnull().any(1)]
df = df.fillna('')
nan_data = df[df.isnull().any(1)]

In [8]:
if os.path.isfile('feature_tm_test.csv'):
    df = pd.read_csv("feature_tm_test.csv",encoding='latin-1')
else:
    def normalized_common_word(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * len(w1 & w2)

    def normalized_total_word(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * (len(w1) + len(w2))

    def normalized_word_share(row):
        w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
        w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
        return 1.0 * len(w1 & w2)/(len(w1) + len(w2))

    def get_2_gram_share(row):
        q1_list = str(row['question1']).lower().split()
        q2_list = str(row['question2']).lower().split()
        q1_2_gram = set([i for i in zip(q1_list, q1_list[1:])])
        q2_2_gram = set([i for i in zip(q2_list, q2_list[1:])])
        shared_2_gram = q1_2_gram.intersection(q2_2_gram)
        if len(q1_2_gram) + len(q2_2_gram) == 0:
            R2gram = 0
        else:
            R2gram = len(shared_2_gram) / (len(q1_2_gram) + len(q2_2_gram))
        return R2gram

    df['ques1_len'] = df['question1'].str.len() 
    df['ques2_len'] = df['question2'].str.len()
    df['len_diff'] = df['ques1_len'] - df['ques2_len']
    
    df['q1_word_len'] = df['question1'].apply(lambda row: len(row.split(" ")))
    df['q2_word_len'] = df['question2'].apply(lambda row: len(row.split(" ")))
    df['words_diff'] = df['q1_word_len'] - df['q2_word_len']
    
    df['q1_caps_count'] = df['question1'].apply(lambda x:sum(1 for i in str(x) if i.isupper()))
    df['q2_caps_count'] = df['question2'].apply(lambda x:sum(1 for i in str(x) if i.isupper()))
    df['caps_diff'] = df['q1_caps_count'] - df['q2_caps_count']
    
    df['q1_char_len'] = df['question1'].apply(lambda x: len(str(x).replace(' ', '')))
    df['q2_char_len'] = df['question2'].apply(lambda x: len(str(x).replace(' ', '')))
    df['diff_char_len'] = df['q1_char_len'] - df['q2_char_len']
    
    df['avg_word_len1'] = df['q1_char_len'] / df['q1_word_len']
    df['avg_word_len2'] = df['q2_char_len'] / df['q2_word_len']
    df['diff_avg_word'] = df['avg_word_len1'] - df['avg_word_len2']
    df['common_word'] = df.apply(normalized_common_word, axis=1)
    df['total_word'] = df.apply(normalized_total_word, axis=1)
    df['word_share'] = df.apply(normalized_word_share, axis=1)
    df['share_2_gram'] = df.apply(get_2_gram_share, axis=1) 



    df.to_csv("feature_tm_test.csv", index=False)

df.head()

,test_id,question1,question2,ques1_len,ques2_len,len_diff,q1_word_len,q2_word_len,words_diff,q1_caps_count,q2_caps_count,caps_diff,q1_char_len,q2_char_len,diff_char_len,avg_word_len1,avg_word_len2,diff_avg_word,common_word,total_word,word_share,share_2_gram
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,57,68,-11,11,14,-3,5,4,1,47,55,-8,4.272727,3.928571,0.344156,2.0,24.0,0.083333,0.043478
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,66,43,23,14,7,7,3,1,2,53,37,16,3.785714,5.285714,-1.500000,4.0,21.0,0.190476,0.105263
2,2,What but is the best way to send money from Ch...,What you send money to China?,60,29,31,14,6,8,4,2,2,47,24,23,3.357143,4.000000,-0.642857,4.0,18.0,0.222222,0.055556
3,3,Which food not emulsifiers?,What foods fibre?,27,17,10,4,3,1,1,1,0,24,15,9,6.000000,5.000000,1.000000,0.0,7.0,0.000000,0.000000
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,32,30,2,4,6,-2,1,2,-1,29,25,4,7.250000,4.166667,3.083333,3.0,10.0,0.300000,0.125000


In [9]:
if os.path.isfile('feature_tm_test.csv'):
    df_tm = pd.read_csv("feature_tm_test.csv",encoding='latin-1')
    print(df_tm.isna().sum())
    df_tm = df_tm.fillna('')
    df_tm.head()
else:
    print("There is no feature_tm_test.csv!")

test_id          0
question1        0
question2        0
ques1_len        0
ques2_len        0
len_diff         0
q1_word_len      0
q2_word_len      0
words_diff       0
q1_caps_count    0
q2_caps_count    0
caps_diff        0
q1_char_len      0
q2_char_len      0
diff_char_len    0
avg_word_len1    0
avg_word_len2    0
diff_avg_word    0
common_word      0
total_word       0
word_share       0
share_2_gram     0
dtype: int64


In [10]:
print(df.isna().sum())

test_id          0
question1        0
question2        0
ques1_len        0
ques2_len        0
len_diff         0
q1_word_len      0
q2_word_len      0
words_diff       0
q1_caps_count    0
q2_caps_count    0
caps_diff        0
q1_char_len      0
q2_char_len      0
diff_char_len    0
avg_word_len1    0
avg_word_len2    0
diff_avg_word    0
common_word      0
total_word       0
word_share       0
share_2_gram     0
dtype: int64


In [11]:
import nltk
nltk.download('stopwords')

SAFE_DIV = 0.0001 


from nltk.corpus import stopwords
STOP_WORDS = stopwords.words('english')

def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    
    return x


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
import gensim

# Download GloVe model
# !wget http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip glove.840B.300d.zip

from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="glove.840B.300d.txt", word2vec_output_file="glove_vectors.txt")

from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("glove_vectors.txt", binary=False)

In [15]:
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, canberra, euclidean, minkowski

def remove_stop(sentence):
    sentence  = str(sentence)
    if sentence == None:
        return ' '
    if sentence == np.nan:
        return ' '
    if sentence == 'NaN':
        return ' '
    z = [i for i in sentence.split() if i not in STOP_WORDS]
    return ' '.join(z)

def wmd(s1, s2, model):
    s1 = str(s1)
    s2 = str(s2)
    s1 = s1.split()
    s2 = s2.split()
    return model.wmdistance(s1, s2)

def g2w2v(list_of_sent, model, d):
    sent_vectors = []
    for sentence in list_of_sent: 
        doc = [word for word in sentence if word in model.wv.vocab] 
        if doc:
            sent_vec = np.mean(model.wv[doc],axis=0) 
        else:
            sent_vec = np.zeros(d)
        sent_vectors.append(sent_vec)
    return sent_vectors

def get_distance_features(df):
    
    print("Extracting Distance Features..")
    
    df['question1'] = df.question1.apply(remove_stop)
    df['question2'] = df.question2.apply(remove_stop)
    df['word_mover_dist'] = df.apply(lambda x: wmd(x['question1'], x['question2'],glove_model), axis=1)
    
    print("- wmd done...")
    
    
    list_of_question1=[]
    for sentence in df.question1.values:
        list_of_question1.append(sentence.split())
    
    list_of_question2=[]
    for sentence in df.question2.values:
        list_of_question2.append(sentence.split())
    
    g2w2v_q1 = g2w2v(list_of_question1, glove_model, 300)
    g2w2v_q2 = g2w2v(list_of_question2, glove_model, 300)
    
    print("- embedding done...")
    
    df['cosine_dist'] = [cosine(q1, q2) for (q1, q2) in zip(g2w2v_q1,g2w2v_q2)]
    df['cityblock_dist'] = [cityblock(q1, q2) for (q1, q2) in zip(g2w2v_q1,g2w2v_q2)]
    df['canberra_dist'] = [canberra(q1, q2) for (q1, q2) in zip(g2w2v_q1,g2w2v_q2)]
    df['euclidean_dist'] = [euclidean(q1, q2) for (q1, q2) in zip(g2w2v_q1,g2w2v_q2)]
    df['minkowski_dist'] = [minkowski(q1, q2) for (q1, q2) in zip(g2w2v_q1,g2w2v_q2)]
    
    print('- spatial distance done')
    
    df.cosine_dist = df.cosine_dist.fillna(0)
    df.word_mover_dist = df.word_mover_dist.apply(lambda wmd: 30 if wmd == np.inf else wmd )
   
    return df

In [16]:
def get_token_features(q1, q2):
    token_features = [0.0]*10
    
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features
    
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    common_word_count = len(q1_words.intersection(q2_words))
    
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
    
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2
    return token_features


def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)


In [17]:
def extract_features(df):
    df["question1"] = df["question1"].fillna("").apply(preprocess)
    df["question2"] = df["question2"].fillna("").apply(preprocess)

    print("Extracting Token Features...")
    
    token_features = df.apply(lambda x: get_token_features(x["question1"], x["question2"]), axis=1)
    
    df["cwc_min"]       = list(map(lambda x: x[0], token_features))
    df["cwc_max"]       = list(map(lambda x: x[1], token_features))
    df["csc_min"]       = list(map(lambda x: x[2], token_features))
    df["csc_max"]       = list(map(lambda x: x[3], token_features))
    df["ctc_min"]       = list(map(lambda x: x[4], token_features))
    df["ctc_max"]       = list(map(lambda x: x[5], token_features))
    df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
    df["first_word_eq"] = list(map(lambda x: x[7], token_features))
    df["abs_len_diff"]  = list(map(lambda x: x[8], token_features))
    df["mean_len"]      = list(map(lambda x: x[9], token_features))
   
    print("Extracting Fuzzy Features..")

    df["token_set_ratio"]       = df.apply(lambda x: fuzz.token_set_ratio(x["question1"], x["question2"]), axis=1)
    df["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
    df["fuzz_ratio"]            = df.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
    df["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
    df["longest_substr_ratio"]  = df.apply(lambda x: get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)
    return df

In [18]:
if os.path.isfile('feature_nlp_test.csv'):
    df_nlp = pd.read_csv("feature_nlp_test.csv",encoding='latin-1')
    # df.fillna('')
else:
    print("Extracting features for test:")
    df = pd.read_csv("new_test.csv")
    df = extract_features(df)
    df = get_distance_features(df)
    df = df.drop(['question1','question2'], axis=1)
    df.to_csv("feature_nlp_test.csv", index=False)
df.head()

Extracting features for test:
Extracting Token Features...
Extracting Fuzzy Features..
Extracting Distance Features..
- wmd done...
- embedding done...
- spatial distance done


,test_id,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,word_mover_dist,cosine_dist,cityblock_dist,canberra_dist,euclidean_dist,minkowski_dist
0,0,0.599988,0.333330,0.000000,0.000000,0.272725,0.214284,0,0,3,12.5,54,50,37,45,0.241379,5.377619,0.212743,32.914707,158.719900,2.450075,2.450075
1,1,0.799984,0.571420,0.499975,0.142855,0.714276,0.357140,0,0,7,10.5,82,58,47,56,0.386364,2.885574,0.082187,22.012440,127.354115,1.689144,1.689144
2,2,0.999967,0.499992,0.666644,0.333328,0.833319,0.357140,0,1,8,10.0,92,55,57,83,0.400000,3.004117,0.094164,26.997807,123.466020,1.951432,1.951432
3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,3.5,52,52,52,53,0.277778,7.164147,0.391609,73.987938,177.060048,5.201015,5.201015
4,4,0.999950,0.666644,0.999900,0.249994,0.749981,0.499992,1,1,2,5.0,74,66,70,73,0.483871,3.298204,0.247688,40.107086,147.543857,3.130887,3.130887
